# 1. Importação das Bibliotecas

Importando tudo o que precisamos: numpy para carregar os dados, pandas para o gráfico, matplotlib para visualização, os modelos e métricas do sklearn, e o BayesSearchCV do scikit-optimize (skopt).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import time

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Bibliotecas importadas com sucesso!")

ModuleNotFoundError: No module named 'importlib.metadata'

# 2. Carregamento dos Dados

Carregando os arquivos .npy que preparamos nas etapas anteriores (treino balanceado com SMOTE e teste original).

In [ ]:
try:
    X_train = np.load("X_train_smote.npy", allow_pickle=True)
    y_train = np.load("y_train_smote.npy", allow_pickle=True)
    X_test = np.load("X_test_final.npy", allow_pickle=True)
    y_test = np.load("y_test.npy", allow_pickle=True)

    print(f"Formato X_train: {X_train.shape}")
    print(f"Formato y_train: {y_train.shape}")
    print(f"Formato X_test: {X_test.shape}")
    print(f"Formato y_test: {y_test.shape}")

except FileNotFoundError:
    print("Erro: Verifique se os arquivos .npy estão no mesmo diretório do notebook.")
    # Criando dados fictícios para o código rodar (apenas para exemplo)
    # X_train, X_test, y_train, y_test = train_test_split(np.random.rand(100, 10), np.random.randint(0, 2, 100), test_size=0.3)
    # print("\nUsando dados fictícios para demonstração.")

# 3. Definição dos Espaços de Busca (Hyperparameters)

Vamos definir os intervalos que o BayesSearchCV irá explorar para cada modelo.

In [ ]:
# Espaço de busca para KNN
search_space_knn = {
    'n_neighbors': Integer(3, 25),
    'weights': Categorical(['uniform', 'distance']),
    'metric': Categorical(['euclidean', 'manhattan', 'minkowski'])
}

# Espaço de busca para Decision Tree
search_space_dt = {
    'criterion': Categorical(['gini', 'entropy']),
    'max_depth': Integer(3, 50),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 20)
}

# Espaço de busca para Random Forest
search_space_rf = {
    'n_estimators': Integer(50, 300),
    'max_depth': Integer(5, 50),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 20),
    'criterion': Categorical(['gini', 'entropy'])
}

# 4. Configuração e Execução do BayesSearchCV

Esta etapa pode demorar! O `n_iter` define quantas combinações de hiperparâmetros serão testadas.
Estamos usando `scoring='f1'` (bom para dados balanceados/desbalanceados) e `cv=5` (validação cruzada de 5 folds).

In [ ]:
# Dicionário para armazenar os melhores modelos
best_models = {}

# --- Otimização KNN ---
print("Iniciando otimização do KNN...")
start_time = time.time()
bayes_knn = BayesSearchCV(
    KNeighborsClassifier(),
    search_space_knn,
    n_iter=30,  # Reduza para 15-20 se for muito lento
    cv=5,
    n_jobs=-1,  # Usar todos os processadores
    scoring='f1',
    random_state=42
)
bayes_knn.fit(X_train, y_train)
best_models['KNN'] = bayes_knn.best_estimator_
print(f"KNN otimizado em {time.time() - start_time:.2f}s")
print(f"Melhores parâmetros KNN: {bayes_knn.best_params_}\n")


# --- Otimização Decision Tree ---
print("Iniciando otimização da Decision Tree...")
start_time = time.time()
bayes_dt = BayesSearchCV(
    DecisionTreeClassifier(random_state=42),
    search_space_dt,
    n_iter=30,
    cv=5,
    n_jobs=-1,
    scoring='f1',
    random_state=42
)
bayes_dt.fit(X_train, y_train)
best_models['Decision Tree'] = bayes_dt.best_estimator_
print(f"Decision Tree otimizada em {time.time() - start_time:.2f}s")
print(f"Melhores parâmetros DT: {bayes_dt.best_params_}\n")


# --- Otimização Random Forest ---
print("Iniciando otimização do Random Forest...")
start_time = time.time()
bayes_rf = BayesSearchCV(
    RandomForestClassifier(random_state=42),
    search_space_rf,
    n_iter=30,
    cv=5,
    n_jobs=-1,
    scoring='f1',
    random_state=42
)
bayes_rf.fit(X_train, y_train)
best_models['Random Forest'] = bayes_rf.best_estimator_
print(f"Random Forest otimizado em {time.time() - start_time:.2f}s")
print(f"Melhores parâmetros RF: {bayes_rf.best_params_}\n")

print("\nOtimização de todos os modelos concluída!")

# 5. Avaliação dos Modelos no Conjunto de Teste

Agora vamos usar os *melhores modelos* encontrados para fazer previsões no conjunto de teste (X_test_final) e calcular as métricas.

In [ ]:
results_list = []

for model_name, model in best_models.items():
    print(f"--- Avaliando: {model_name} ---")
    
    # Fazer previsões
    y_pred = model.predict(X_test)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Imprimir o relatório de classificação
    print(classification_report(y_test, y_pred))
    
    # Guardar resultados
    results_list.append({
        'Modelo': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

# Converter lista de resultados para DataFrame
df_results = pd.DataFrame(results_list)
df_results = df_results.set_index('Modelo')

print("\n--- DataFrame de Resultados ---")
print(df_results)

# 6. Visualização Comparativa das Métricas

Finalmente, o gráfico de barras horizontal para comparar o desempenho dos modelos.

In [ ]:
# Reformatando o DataFrame para facilitar o plot (melt)
df_plot = df_results.stack().reset_index()
df_plot.columns = ['Modelo', 'Métrica', 'Valor']

# Criando o gráfico de barras horizontal (agrupado)
plt.figure(figsize=(14, 8))
ax = sns.barplot(
    data=df_plot,
    y='Métrica',
    x='Valor',
    hue='Modelo',
    orient='h'
)

# Adicionando título e ajustando legendas
ax.set_title('Comparação de Métricas dos Modelos (Dados de Teste)', fontsize=16, fontweight='bold')
ax.set_xlabel('Pontuação', fontsize=12)
ax.set_ylabel('Métrica', fontsize=12)
ax.set_xlim(0, 1.05) # Limite do eixo X (métrica vai de 0 a 1)

# Colocar valores nas barras
for p in ax.patches:
    width = p.get_width()
    ax.text(width + 0.01, 
            p.get_y() + p.get_height() / 2, 
            f'{width:.3f}', 
            va='center')

plt.legend(title='Modelo', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()